# K-Means Clustering Evaluation

This Python notebook is used for evaluation of a dictionary that is produced by:

1. Clustering word vectors by K-Means clustering
2. Converting into a dictionary that corresponds to 'Word: Index' pairing

The evaluation consists of:

- Find the cluster a word belongs to 
- Find the other words in the cluster for a specific word

In [3]:
import pickle

# Specify the files
FILE_DICT = "C:/Users/MyPC/Desktop/FYP/Word Dictionaries/dict_500C.pk"
FILE_CLUS = "C:/Users/MyPC/Desktop/FYP/K-Means Models/full_500C.pk"

# Load using pickle
array_dict_cluster = pickle.load(open(FILE_DICT, "rb"))
word_centroid_map =  pickle.load(open(FILE_CLUS,"rb"))

total_clusters = max(word_centroid_map.values()) + 1
average_word = round(len(word_centroid_map)/total_clusters)

# Display results
print("TOTAL WORDS: %i" % (len(word_centroid_map)))
print("TOTAL CLUSTERS USED: %i " % (total_clusters))
print("AVERAGE PER CLUSTER %i" % (average_word))

TOTAL WORDS: 1146604
TOTAL CLUSTERS USED: 500 
AVERAGE PER CLUSTER 2293


In [8]:
# Find the cluster of words, based on a given word
search = "convolutions"

# Get the key, or cluster number
cluster_num = word_centroid_map[search]

# Return the array based on the cluster number
words = array_dict_cluster[cluster_num]['word_list']
        
# Get index number of word (VERIFICATION)
index_search = words.index(search)

# Display results
print("CLUSTER NUMBER: %i" % (cluster_num))
print("SEARCHED WORD: %s \n" % (search))

print("INDEX NUMBER: %i" % (index_search))
print("words[%i] = %s \n" % (index_search, words[index_search]))

print("TOTAL WORDS: %i \n" % (len(words)))

print("WORDS: ", words[:200])

CLUSTER NUMBER: 124
SEARCHED WORD: convolutions 

INDEX NUMBER: 11
words[11] = convolutions 

TOTAL WORDS: 1694 

WORDS:  ['commutation', 'distributive', 'bayesian', 'endogeneity', 'matricies', 'associativity', 'compactness', 'hermitian', 'numerator', 'submanifolds', 'uncountable', 'convolutions', 'invariant', 'polytope', 'theorical', 'overconstrained', 'eigendecomposition', 'observationally', 'boundedness', 'discriminant', 'brzozowski', 'tangency', 'parametrization', 'entropies', 'wasserstein', 'precalculated', 'baconian', 'asymptote', 'congruential', 'visualised', 'permuting', 'arccos', 'subranges', 'spacelike', 'factorable', 'kinematics', 'homography', 'riemman', 'renormalizability', 'theoy', 'method', 'autoregression', 'lognormal', 'superposition', 'quantization', 'octet', 'renormalizable', 'relativities', 'pseudovector', 'sines', 'microarrays', 'hashlife', 'summands', 'probabilty', 'nabla', 'monomial', 'sinusoidal', 'borromean', 'transversality', 'pythagorus', 'transitivity', 'mal